In [1]:
import pandas as pd
df = pd.read_csv('NewFeatures.csv') #NewFeatures.csv has trucks labeled as 1 for failure and 2 for nonfailures 15-30 days after service


In [2]:
df['Failure'].value_counts()

0    108971
1      1380
2      1356
Name: Failure, dtype: int64

In [3]:
df.head()

,Unnamed: 0,date,platform_id,distance_miles,fuel_used_gallons,duration_mins,idle_duration_mins,fuel_loss_gallons,dpf_regen_inhibited_duration_mins,dpf_regen_not_inhibited_duration_mins,...,percent_regen_inhibited,percent_regen_needed,percent_regen_inhibit_switch_not_active,fuels_used_per_mile,miles_per_minutes,mins_idle_per_mile,fuel_lost_per_mile,min_regen_inhibited_per_mile,min_regen_needed_per_mile,min_regen_inhibit_switch_not_active_per_mile
0,0,2019-01-02,309450,65.294602,7.743177,222.002017,44.141633,0.057875,209.524550,11.435950,...,0.943796,0.000000,0.0,0.118588,0.294117,0.676038,0.000886,3.208911,0.000000,0.0
1,1,2019-01-03,309450,210.269609,32.266755,519.221017,134.957617,0.495596,462.685000,54.311850,...,0.891114,0.000000,0.0,0.153454,0.404971,0.641831,0.002357,2.200437,0.000000,0.0
2,2,2019-01-04,309450,98.413486,15.543700,354.276533,83.660883,0.281884,319.565517,33.708700,...,0.902023,0.311246,0.0,0.157943,0.277787,0.850096,0.002864,3.247172,1.120448,0.0
3,3,2019-01-05,309450,74.838556,13.602637,131.153333,30.391967,0.225600,115.578267,15.441733,...,0.881245,0.022622,0.0,0.181760,0.570619,0.406100,0.003014,1.544368,0.039644,0.0
4,4,2019-01-06,309450,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df[df['distance_miles']!= 0]

In [5]:
only1and2 = df[df['Failure']!= 0]

In [6]:
only1and2 = only1and2.drop(['dpf_regen_inhibit_switch_not_active_duration_mins', 'min_regen_inhibit_switch_not_active_per_mile'], axis = 1)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


X=only1and2[['percent_fuel_lost','fuels_used_per_mile',
             'miles_per_minutes', 'mins_idle_per_mile', 'fuel_lost_per_mile']]

y=only1and2['Failure']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf=RandomForestClassifier(n_estimators=200)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))

Accuracy: 0.5474956822107081
Recall: 0.6014492753623188
Precision: 0.5220125786163522


In [8]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

miles_per_minutes      0.212897
fuels_used_per_mile    0.208518
percent_fuel_lost      0.193991
fuel_lost_per_mile     0.193292
mins_idle_per_mile     0.191302
dtype: float64